In [29]:
import os
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import ast
import random
plt.style.use('seaborn-white')

while Path.cwd().name != 'cartography':
    %cd ..

In [30]:
DATA_DIR = Path('data/glue/MNLI/')

## split data into train.tsv and unlabeled.tsv for active learning

In [31]:
with open(DATA_DIR / 'train.tsv') as fin:
    lines = fin.readlines()

header = lines[0]
lines = lines[1:]
random.shuffle(lines)

In [32]:
train_frac = 0.10
train_size = len(lines)
train_n = int(train_frac * train_size)

with open(DATA_DIR / 'active_learning' / 'train.tsv', 'w') as fo:
    fo.write(header)
    for line in lines[:train_n]:
        fo.write(line)

with open(DATA_DIR / 'active_learning' / 'unlabeled.tsv', 'w') as fo:
    fo.write(header)
    for line in lines[train_n:]:
        fo.write(line)

## create test file with GPT3 output

In [28]:
it = 2
metrics = ['random']#, 'final_confidence', 'mean_variability']
for metric in metrics:
    with open(DATA_DIR / f'active_learning/{metric}/{it}/cartography_{metric}_19635/train.tsv', 'w') as fo, open(DATA_DIR / 'active_learning/train.tsv') as train, open(DATA_DIR / f'active_learning/{metric}/{it}/cartography_{metric}_19635/selected.tsv') as selected:
        train_lines = train.readlines()
        selected_lines = selected.readlines()[1:]
        train_lines.extend(selected_lines)
        for line in train_lines:
            fo.write(line)

In [77]:
gpt3_examples = [
    {
        'sentence1': "I very rarely stop in at The Gun Shop in Reno, NV.",
        'sentence2': "I always make it a point to stop in when I'm in town, and I've never been disappointed.",
        'gold_label': 'contradiction'
    },
    {
        'sentence1': "Everybody had rushed to help, but it was too late. ",
        'sentence2': "The old man had died.",
        'gold_label': 'neutral'
    },
    {
        'sentence1': "Blatter suggested that immediate reviews of television evidence should be allowed in cases where the referee has made a mistake in awarding a penalty. “If a referee makes a mistake, then he should be given the opportunity to see it again and give the right decision,” Blatter said.",
        'sentence2': "“If a referee makes a mistake in awarding a penalty, then he should be allowed to go back and see it again.”",
        'gold_label': 'entailment'
    },
    {
        'sentence1': "The boba was too sweet, but the drink was good!",
        'sentence2': "The teas are great and the boba are even better!",
        'gold_label': 'contradiction'
    },
    {
        'sentence1': "I don’t know where to start.",
        'sentence2': "The question of what is the role of the federal government in education is just a very difficult question.",
        'gold_label': 'neutral'
    },
    {
        'sentence1': "I don’t know where to start. The question of what is the role of the federal government in education is just a very difficult question.",
        'sentence2': "It’s one of the most difficult questions we face in government.",
        'gold_label': 'neutral'
    },
    {
        'sentence1': "Secondly, that workers should see beyond their various national and religious differences and unite.",
        'sentence2': "This meant overcoming the divide-and-rule policies of their employers.",
        'gold_label': 'neutral'
    },
    {
        'sentence1': "Secondly, that workers should see beyond their various national and religious differences and unite. This meant overcoming the divide-and-rule policies of their employers.",
        'sentence2': "And thirdly, that they should organise themselves in trade-union-like organisations and struggle collectively for their rights.",
        'gold_label': 'neutral'
    },
    {
        'sentence1': "He later made attempts to become the first black mayor of Omaha, and ran for Lieutenant Governor of Nebraska.",
        'sentence2': "He was also a candidate for the United States House of Representatives.",
        'gold_label': 'neutral',
    },
    {
        'sentence1': "He later made attempts to become the first black mayor of Omaha, and ran for Lieutenant Governor of Nebraska. He was also a candidate for the United States House of Representatives.",
        'sentence2': "He died at the age of 22 in a car accident.",
        'gold_label': 'contradiction'
    },
    {
        'sentence1': "He later made attempts to become the first black mayor of Omaha, and ran for Lieutenant Governor of Nebraska. He was also a candidate for the United States House of Representatives.",
        'sentence2': "He died at the age of 45 in a car accident.",
        'gold_label': 'neutral'
    },
    {
        'sentence1': "Unsurprisingly, Pence also has some backward religious beliefs.",
        'sentence2': "He refuses to meet alone with women other than his wife, for example, and he thinks Jesus would be opposed to abortion rights.",
        'gold_label': 'neutral'
    },
    {
        'sentence1': "Labour is now the largest political party in western Europe.",
        'sentence2': "Labour is now the largest political party in western Europe.",
        'gold_label': 'entailment'
    },
    {
        'sentence1': "De Blasio was not the only mayor representing his city at the march.",
        'sentence2': "The mayors of Boston, Los Angeles, Baltimore, and many others were also in attendance.",
        'gold_label': 'neutral'
    },
    {
        'sentence1': "De Blasio was not the only mayor representing his city at the march.",
        'sentence2': "The mayors of New York, Los Angeles, Baltimore, and many others were also in attendance.",
        'gold_label': 'contradiction'
    },
    {
        'sentence1': "Private contractors, previously immune to prosecution in Iraq, are now subject to Iraqi law, and will be tried in Iraqi courts. The Iraqi government has now taken over the program, and is using it to track down former members of Saddam Hussein's government.",
        'sentence2': "The Iraqi government is now administering the program, and is using it to track down former members of Saddam Hussein's government.",
        'gold_label': 'entailment'
    },
    {
        'sentence1': "Japan, Korea, China and Indonesia farm seaweed and process it into a product that is safe to eat.",
        'sentence2': "But in Australia, where only two commercial farms produce the crop, there are no safety standards and no controls on how the seaweed is farmed or processed.",
        'gold_label': 'neutral'
    },
    {
        'sentence1': "Japan, Korea, China and Indonesia farm seaweed and process it into a product that is safe to eat. But in Australia, where only two commercial farms produce the crop, there are no safety standards and no controls on how the seaweed is farmed or processed.",
        'sentence2': "According to a recent study by the University of Tasmania, almost half of the seaweed samples tested from local supermarkets contained traces of heavy metals.",
        'gold_label': 'neutral'
    },
    {
        'sentence1': "The missing flight triggered many countries to join hands and work together to find the missing airliner.",
        'sentence2': "The search for the missing flight is still on.",
        'gold_label': 'neutral'
    },
    {
        'sentence1': "For instance, let's say that you and your coworker get into an argument over an issue. The coworker is upset and you're upset.",
        'sentence2': "Your coworker is upset and you're upset.",
        'gold_label': 'entailment'
    },
    {
        'sentence1': "For instance, let's say that you and your coworker get into an argument over an issue. The coworker is upset and you're upset. That's the part I want to talk about.",
        'sentence2': "Your coworker is upset and you're upset.",
        'gold_label': 'neutral'
    },
    {
        'sentence1': "A boat sinked in the Pacific ocean. The captain was able to escape. He spent 40 days on a raft with a few others. They were finally picked up. They were shocked to see there was a two-year-old boy on board.",
        'sentence2': "Even more shocking was that he had been born on the raft during the 40 days.",
        'gold_label': 'contradiction'
    },
    {
        'sentence1': "A boat sinked in the Pacific ocean. Its captain drowned. A large catfish swallowed the corpse.",
        'sentence2': "After some time, the catfish was caught by a fisherman and send to the market.",
        'gold_label': 'neutral',
    },
    {
        'sentence1': "A boat sinked in the Pacific ocean. Its captain drowned. A large catfish swallowed the corpse.",
        'sentence2': "After some time, the catfish was caught by the captain who was in the boat in the Pacific ocean.",
        'gold_label': 'contradiction'
    }
]

In [78]:
FIELDS = ["index", "promptID", "pairID", "genre", "sentence1_binary_parse", "sentence2_binary_parse", "sentence1_parse", "sentence2_parse", "sentence1", "sentence2", "gold_label"]

with open(DATA_DIR / 'gpt3_test/test.tsv', 'w') as fo:
    fo.write('\t'.join(FIELDS) + '\n')
    for i, ex in enumerate(gpt3_examples):
        ex['index'] = str(i)
        out_fields = [ex[field] if field in ex else "" for field in FIELDS]
        fo.write("\t".join(out_fields) + "\n")

In [79]:
gpt3_examples_df = pd.read_csv(DATA_DIR / 'gpt3_test/test.tsv', sep='\t')
idx2label = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}

In [80]:
with open('output/mnli/test_dynamics/dynamics_epoch_None.jsonl') as fin:
    for line in fin:
        line = ast.literal_eval(line)
        idx = (line['guid'] - 3) // 10
        print(f'Hypothesis: {gpt3_examples_df.at[idx, "sentence1"]}')
        print(f'Premise: {gpt3_examples_df.at[idx, "sentence2"]}')
        print(f'Gold: {gpt3_examples_df.at[idx, "gold_label"]}')
        print(f'Label: {idx2label[np.argmax(line["logits_epoch_None"])]}\n')

Hypothesis: I very rarely stop in at The Gun Shop in Reno, NV.
Premise: I always make it a point to stop in when I'm in town, and I've never been disappointed.
Gold: contradiction
Label: contradiction

Hypothesis: Everybody had rushed to help, but it was too late. 
Premise: The old man had died.
Gold: neutral
Label: neutral

Hypothesis: Blatter suggested that immediate reviews of television evidence should be allowed in cases where the referee has made a mistake in awarding a penalty. “If a referee makes a mistake, then he should be given the opportunity to see it again and give the right decision,” Blatter said.
Premise: “If a referee makes a mistake in awarding a penalty, then he should be allowed to go back and see it again.”
Gold: entailment
Label: entailment

Hypothesis: The boba was too sweet, but the drink was good!
Premise: The teas are great and the boba are even better!
Gold: contradiction
Label: contradiction

Hypothesis: I don’t know where to start.
Premise: The question of